In [2]:
#After executing the cell,please RESTART the kernel and run all the cells.
!pip install "ibm-watsonx-ai==1.0.10"
!pip install "langchain==0.2.6" 
!pip install "langchain-ibm==0.1.8"
!pip install "langchain-community==0.2.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 8.4 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [956 lines of output]
      + meson setup /private/var/folders/kk/hlh79rdj4gzb9s7r5s_z34lm0000gn/T/pip-install-x1v16mm_/pandas_4b9306c1509f47578ec0111ff76a4295 /private/var/folders/kk/hlh79rdj4gzb9s7r5s_z34lm0000gn/T/pip-install-x1v16mm_/pandas_4b9306c1509f47578ec0111ff76a4295/.mesonpy-zzmbitnt/build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --vsenv --native-file=/private/var/folders/kk/hlh79rdj4gzb9s7r5s_z34lm0000gn/T/pip-install-x1v16mm_/pandas_4b9306c1509f47578ec0111ff76a4295/.mesonpy-zzmbitnt/build/meson-python-native-file.ini
      The Meson build system
      Version: 1.2.1
    

In [1]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxLLM

## Build LLM

Here, you will create a function that interacts with the watsonx.ai API, enabling you to utilize various models available.

You just need to input the model ID in string format, then it will return you with the LLM object. You can use it to invoke any queries. A list of model IDs can be found in [here](https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html).


In [4]:
def llm_model(model_id):
    parameters = {
        GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
        GenParams.TEMPERATURE: 0.5, # this randomness or creativity of the model's responses
    }
    
    credentials = {
        "url": "https://us-south.ml.cloud.ibm.com"
    }
    
    project_id = "skills-network"
    
    model = ModelInference(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        project_id=project_id
    )
    
    llm = WatsonxLLM(watsonx_model = model)
    return llm

In [8]:
# llama_llm = llm_model('meta-llama/llama-3-3-70b-instruct')
# llama_llm.invoke("How are you?")

## Load source document
A document has been prepared here.

In [9]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/d_ahNwb1L2duIxBR6RD63Q/state-of-the-union.txt"

--2025-09-07 17:40:03--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/d_ahNwb1L2duIxBR6RD63Q/state-of-the-union.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state-of-the-union.txt’

state-of-the-union. 100%[===================>]  38.11K  61.7KB/s    in 0.6s    

2025-09-07 17:40:06 (61.7 KB/s) - ‘state-of-the-union.txt’ saved [39027/39027]



In [10]:
loader = TextLoader("state-of-the-union.txt")
data = loader.load()
content = data[0].page_content
content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

## Limitation of retrieve directly from full document

Before you explore the limitations of directly retrieving information from a full document, you need to understand a concept called `context length`. 

`Context length` in LLMs refers to the amount of text or information (prompt) that the model can consider when processing or generating output. LLMs have a fixed context length, meaning they can only take into account a limited amount of text at a time.


So, how long is your source document here? The answer is 8,235 tokens, which you calculated using this [platform](https://platform.openai.com/tokenizer).


### LangChain prompt template

A prompt template has been set up using LangChain to make it reusable.

In this template, you will define two input variables:
- `content`: This variable will hold all the content from the entire source document at once.
- `question`: This variable will capture the user's query.


In [11]:
template = """According to the document content here 
            {content},
            answer this question 
            {question}.
            Do not try to make up the answer.
                
            YOUR RESPONSE:
"""

prompt_template = PromptTemplate(template=template, input_variables=['content', 'question'])
prompt_template 

PromptTemplate(input_variables=['content', 'question'], template='According to the document content here \n            {content},\n            answer this question \n            {question}.\n            Do not try to make up the answer.\n\n            YOUR RESPONSE:\n')

### Use mixtral model
Since the context window length of the mixtral model is longer than your source document, you can assume it can retrieve relevant information for the query when you input the whole document into the prompt.

First, let's build a mixtral model.


In [15]:
mixtral_llm = llm_model('meta-llama/llama-4-maverick-17b-128e-instruct-fp8')
query_chain = LLMChain(llm=mixtral_llm, prompt=prompt_template)

query = "It is in which year of our nation?"
response = query_chain.invoke(input={'content': content, 'question': query})
print(response['text'])

Ypu have asked a question whose answer appears at the very end of the document. Despite this, the LLM was still able to answer it correctly because the model's context window is long enough to accommodate the entire content of the document.


### Use Llama 3 model

Now, let's try using Llama3 model.

First, create a query chain.


In [16]:
query_chain = LLMChain(llm=llama_llm, prompt=prompt_template)
query_chain 

NameError: name 'llama_llm' is not defined

In [ ]:
query = "It is in which year of our nation?"
response = query_chain.invoke(input={'content': content, 'question': query})
print(response['text'])

#### Take away

If the document is much longer than the LLM's context length, it is important and necessary to cut the document into chunks, index them, and then let the LLM retrieve the relevant information accurately and efficiently.

